In [1]:
import gym
import numpy as np
import torch
from utils import *
import torch
import torch.nn as nn

In [2]:
# Run this cell without modification

def evaluate(policy, num_episodes=1, seed=0, env_name='FrozenLake8x8-v1',
             render=False, existing_env=None):
    """This function evaluate the given policy and return the mean episode 
    reward.
    :param policy: a function whose input is the observation
    :param num_episodes: number of episodes you wish to run
    :param seed: the random seed
    :param env_name: the name of the environment
    :param render: a boolean flag indicating whether to render policy
    :return: the averaged episode reward of the given policy.
    """
    if existing_env is None:
        env = gym.make(env_name)
        env.seed(seed)
    else:
        env = existing_env
    rewards = []
    if render: num_episodes = 1
    for i in range(num_episodes):
        obs = env.reset()
        act = policy(obs)
        ep_reward = 0
        while True:
            obs, reward, done, info = env.step(act)
            act = policy(obs)
            ep_reward += reward
            if render:
                env.render()
                wait(sleep=0.05)
            if done:
                break
        rewards.append(ep_reward)
    if render:
        env.close()
    return np.mean(rewards)

In [3]:
# Run this cell without modification

def run(trainer_cls, config=None, reward_threshold=None):
    """Run the trainer and report progress, agnostic to the class of trainer
    :param trainer_cls: A trainer class 
    :param config: A dict
    :param reward_threshold: the reward threshold to break the training
    :return: The trained trainer and a dataframe containing learning progress
    """
    if config is None:
        config = {}
    trainer = trainer_cls(config)
    config = trainer.config
    start = now = time.time()
    stats = []
    for i in range(config['max_iteration'] + 1):
        stat = trainer.train()
        stats.append(stat or {})
        if i % config['evaluate_interval'] == 0 or \
                i == config["max_iteration"]:
            reward = trainer.evaluate(config.get("evaluate_num_episodes", 50))
            print("({:.1f}s,+{:.1f}s)\tIteration {}, current mean episode "
                  "reward is {}. {}".format(
                time.time() - start, time.time() - now, i, reward,
                {k: round(np.mean(v), 4) for k, v in
                 stat.items()} if stat else ""))
            now = time.time()
        if reward_threshold is not None and reward > reward_threshold:
            print("In {} iteration, current mean episode reward {:.3f} is "
                  "greater than reward threshold {}. Congratulation! Now we "
                  "exit the training process.".format(
                i, reward, reward_threshold))
            break
    return trainer, stats

In [4]:
# Solve TODOs and remove "pass"

default_config = dict(
    env_name="CartPole-v0",
    max_iteration=1000,
    max_episode_length=1000,
    evaluate_interval=100,
    gamma=0.99,
    eps=0.3,
    seed=0
)


class AbstractTrainer:
    """This is the abstract class for value-based RL trainer. We will inherent
    the specify algorithm's trainer from this abstract class, so that we can
    reuse the codes.
    """

    def __init__(self, config):
        self.config = merge_config(config, default_config)

        # Create the environment
        self.env_name = self.config['env_name']
        self.env = gym.make(self.env_name)
        if self.env_name == "Pong-ram-v0":
            self.env = wrap_deepmind_ram(self.env)

        # Apply the random seed
        self.seed = self.config["seed"]
        np.random.seed(self.seed)
        self.env.seed(self.seed)

        # We set self.obs_dim to the number of possible observation
        # if observation space is discrete, otherwise the number
        # of observation's dimensions. The same to self.act_dim.
        if isinstance(self.env.observation_space, gym.spaces.box.Box):
            assert len(self.env.observation_space.shape) == 1
            self.obs_dim = self.env.observation_space.shape[0]
            self.discrete_obs = False
        elif isinstance(self.env.observation_space,
                        gym.spaces.discrete.Discrete):
            self.obs_dim = self.env.observation_space.n
            self.discrete_obs = True
        else:
            raise ValueError("Wrong observation space!")

        if isinstance(self.env.action_space, gym.spaces.box.Box):
            assert len(self.env.action_space.shape) == 1
            self.act_dim = self.env.action_space.shape[0]
        elif isinstance(self.env.action_space, gym.spaces.discrete.Discrete):
            self.act_dim = self.env.action_space.n
        else:
            raise ValueError("Wrong action space!")

        self.eps = self.config['eps']

        # You need to setup the parameter for your function approximator.
        self.initialize_parameters()

    def initialize_parameters(self):
        self.parameters = None
        raise NotImplementedError(
            "You need to override the "
            "Trainer._initialize_parameters() function.")

    def process_state(self, state):
        """Preprocess the state (observation).

        If the environment provides discrete observation (state), transform
        it to one-hot form. For example, the environment FrozenLake-v0
        provides an integer in [0, ..., 15] denotes the 16 possible states.
        We transform it to one-hot style:

        original state 0 -> one-hot vector [1, 0, 0, 0, 0, 0, 0, 0, ...]
        original state 1 -> one-hot vector [0, 1, 0, 0, 0, 0, 0, 0, ...]
        original state 15 -> one-hot vector [0, ..., 0, 0, 0, 0, 0, 1]

        If the observation space is continuous, then you should do nothing.
        """
        if not self.discrete_obs:
            return state
        else:
            new_state = np.zeros((self.obs_dim,))
            new_state[state] = 1
        return new_state

    def compute_values(self, processed_state):
        """Approximate the state value of given state.
        This is a private function.
        Note that you should NOT preprocess the state here.
        """
        raise NotImplementedError("You need to override the "
                                  "Trainer.compute_values() function.")

    def compute_action(self, processed_state, eps=None):
        """Compute the action given the state. Note that the input
        is the processed state."""

        values = self.compute_values(processed_state)
        assert values.ndim == 1, values.shape

        if eps is None:
            eps = self.eps

        # [TODO] Implement the epsilon-greedy policy here. We have `eps`
        #  probability to choose a uniformly random action in action_space,
        #  otherwise choose action that maximizes the values.
        # Hint: Use the function of self.env.action_space to sample random
        # action.
        if np.random.uniform(0,1)<self.eps:
            return np.random.choice(self.env.action_space.n)
        else:
            return np.argmax(values)

    def evaluate(self, num_episodes=50, *args, **kwargs):
        """Use the function you write to evaluate current policy.
        Return the mean episode reward of 50 episodes."""
        policy = lambda raw_state: self.compute_action(
            self.process_state(raw_state), eps=0.0)
        if "MetaDrive" in self.env_name:
            kwargs["existing_env"] = self.env
        result = evaluate(policy, num_episodes, seed=self.seed,
                          env_name=self.env_name, *args, **kwargs)
        return result

    def compute_gradient(self, *args, **kwargs):
        """Compute the gradient."""
        raise NotImplementedError(
            "You need to override the Trainer.compute_gradient() function.")

    def apply_gradient(self, *args, **kwargs):
        """Compute the gradient"""
        raise NotImplementedError(
            "You need to override the Trainer.apply_gradient() function.")

    def train(self):
        """Conduct one iteration of learning."""
        raise NotImplementedError("You need to override the "
                                  "Trainer.train() function.")

In [5]:
# Run this cell without modification

class TestTrainer(AbstractTrainer):
    """This class is used for testing. We don't really train anything."""
    def compute_values(self, state):
        return np.random.random_sample(size=self.act_dim)
    def initialize_parameters(self):
        self.parameters = np.random.random_sample(size=(self.obs_dim, self.act_dim))
    
t = TestTrainer(dict(env_name="CartPole-v0"))
obs = t.env.observation_space.sample()
processed = t.process_state(obs)
assert processed.shape == (4, )
assert np.all(processed == obs)
# Test compute_action
values = t.compute_values(processed)
correct_act = np.argmax(values)
assert t.compute_action(processed, eps=0) == correct_act
print("Average episode reward for a random policy in 500 episodes in CartPole-v0: ",
      t.evaluate(num_episodes=500))

Average episode reward for a random policy in 500 episodes in CartPole-v0:  22.554


In [6]:
# Solve the TODOs and remove `pass`

# Build the algorithm-specify config.
linear_approximator_config = merge_config(dict(
    parameter_std=0.01,
    learning_rate=0.01,
    n=3,
), default_config)


class LinearTrainer(AbstractTrainer):
    def __init__(self, config):
        config = merge_config(config, linear_approximator_config)

        # Initialize the abstract class.
        super().__init__(config)

        self.max_episode_length = self.config["max_episode_length"]
        self.learning_rate = self.config["learning_rate"]
        self.gamma = self.config["gamma"]
        self.n = self.config["n"]

    def initialize_parameters(self):
        # [TODO] Initialize self.parameters, which is two dimensional matrix,
        #  and subjects to a normal distribution with scale
        #  config["parameter_std"].
        std = self.config["parameter_std"]
        self.parameters = std*np.random.random_sample(size=(self.obs_dim, self.act_dim))
        
        print("Initialize parameters with shape: {}.".format(
            self.parameters.shape))

    def compute_values(self, processed_state):
        # [TODO] Compute the value for each potential action. Note that you
        #  should NOT preprocess the state here."""
        assert processed_state.ndim == 1, processed_state.shape
        
        ret = np.dot(self.parameters.transpose(),processed_state.transpose())
        
        return ret

    def train(self):
        """
        Please implement the n-step Sarsa algorithm presented in Chapter 10.2
        of the textbook. You algorithm should reduce the convention one-step
        Sarsa when n = 1. That is:
            TD = r_t + gamma * Q(s_t+1, a_t+1) - Q(s_t, a_t)
            Q(s_t, a_t) = Q(s_t, a_t) + learning_rate * TD
        """
        s = self.env.reset()
        processed_s = self.process_state(s)
        processed_states = [processed_s]
        rewards = [0.0]
        actions = [self.compute_action(processed_s)]
        T = float("inf")

        for t in range(self.max_episode_length):
            if t < T:
                # [TODO]  When the termination is not reach, apply action,
                #  process state, record state / reward / action to the
                #  lists defined above, and deal with termination.
                next_state, reward, done, info  = self.env.step(actions[t])
                                
                processed_s = self.process_state(next_state)
                processed_states.append(processed_s)
                rewards.append(reward)
                if done:
                    break
                
                else:
                    next_act = self.compute_action(processed_s)
                    actions.append(next_act)

            tau = t - self.n + 1
            if tau >= 0:
                gradient = self.compute_gradient(
                    processed_states, actions, rewards, tau, T
                )
                self.apply_gradient(gradient)
            if tau == T - 1:
                break

    def compute_gradient(self, processed_states, actions, rewards, tau, T):
        """Compute the gradient"""
        n = self.n

        # [TODO] Compute the approximation goal, the truth state action value
        #  G. It is a n-step discounted sum of rewards. Refer to Chapter 10.2
        #  of the textbook.
        # [HINT] G have two parts: the accumuted reward computed from step tau to 
        #  step tau+n, and the possible state value at time step tau+n, if the episode
        #  is not terminated. Remember to apply the discounter factor (\gamma^n) to
        #  the second part of G if applicable.
        G = 0
        for t in reversed(range(n)):
            G = (G**self.gamma)+rewards[t+tau]
                
        if tau + n < T:
            # [TODO] If at time step tau + n the episode is not terminated,
            # then we should add the state action value at tau + n
            # to the G.
            G += (self.gamma**n)*max(self.compute_values(processed_states[tau+n]))
        
        # Denote the state-action value function Q, then the loss of
        # prediction error w.r.t. the weights can be separated into two
        # parts (the chain rule):
        #     dLoss / dweight = (dLoss / dQ) * (dQ / dweight)
        # We call the first one loss_grad, and the latter one
        # value_grad. We consider the Mean Square Error between the target
        # value (G) and the predicted value (Q(s_t, a_t)) to be the loss.

        loss_grad = np.zeros((self.act_dim, 1))
        # [TODO] fill the propoer value of loss_grad, denoting the gradient
        # of the MSE w.r.t. the output of the linear function.
        loss_grad[:,0] = -(G*np.ones(2)-self.compute_values(processed_states[tau]))

        # [TODO] compute the value of value_grad, denoting the gradient of
        # the output of the linear function w.r.t. the parameters.
        value_grad = np.zeros((self.obs_dim, 1))
        value_grad[:,0] = processed_states[tau]

        assert loss_grad.shape == (self.act_dim, 1)
        assert value_grad.shape == (self.obs_dim, 1)

        # [TODO] merge two gradients to get the gradient of loss w.r.t. the
        # parameters.
        gradient = np.dot(value_grad,loss_grad.transpose())
        pass
    
        return gradient

    def apply_gradient(self, gradient):
        """Apply the gradient to the parameter."""
        assert gradient.shape == self.parameters.shape, (
            gradient.shape, self.parameters.shape)
        # [TODO] apply the gradient to self.parameters
        self.parameters -= self.learning_rate*gradient

In [7]:
# Run this cell without modification

# Build the test trainer.
test_trainer = LinearTrainer(dict(parameter_std=0.0))

# Test self.parameters.
assert test_trainer.parameters.std() == 0.0, \
    "Parameters should subjects to a normal distribution with standard " \
    "deviation config['parameter_std'], but you have {}." \
    "".format(test_trainer.parameters.std())
assert test_trainer.parameters.mean() == 0, \
    "Parameters should subjects to a normal distribution with mean 0. " \
    "But you have {}.".format(test_trainer.parameters.mean())

# Test compute_values
fake_state = test_trainer.env.observation_space.sample()
processed_state = test_trainer.process_state(fake_state)
assert processed_state.shape == (test_trainer.obs_dim, ), processed_state.shape
values = test_trainer.compute_values(fake_state)
assert values.shape == (test_trainer.act_dim, ), values.shape
#import pdb; pdb.set_trace()

# Test compute_gradient
tmp_gradient = test_trainer.compute_gradient(
    [processed_state]*10, [test_trainer.env.action_space.sample()]*10, [0.0]*10, 2, 5)
assert tmp_gradient.shape == test_trainer.parameters.shape

test_trainer.train()
print("Now your codes should be bug-free.")

Initialize parameters with shape: (4, 2).
Now your codes should be bug-free.


In [8]:
# Run this cell without modification

linear_trainer, _ = run(LinearTrainer, dict(
    max_iteration=10000,
    evaluate_interval=1000, 
    parameter_std=0.01,
    learning_rate=0.01,
    n=3,
    env_name="CartPole-v0"
))

# It's OK to see bad performance

Initialize parameters with shape: (4, 2).
(0.2s,+0.2s)	Iteration 0, current mean episode reward is 145.16. 
(2.1s,+1.9s)	Iteration 1000, current mean episode reward is 21.94. 
(3.0s,+0.9s)	Iteration 2000, current mean episode reward is 20.86. 
(3.9s,+0.9s)	Iteration 3000, current mean episode reward is 19.14. 
(4.5s,+0.7s)	Iteration 4000, current mean episode reward is 11.42. 
(5.1s,+0.5s)	Iteration 5000, current mean episode reward is 11.5. 
(5.6s,+0.5s)	Iteration 6000, current mean episode reward is 11.84. 
(6.2s,+0.6s)	Iteration 7000, current mean episode reward is 11.12. 
(6.9s,+0.7s)	Iteration 8000, current mean episode reward is 11.44. 
(7.7s,+0.8s)	Iteration 9000, current mean episode reward is 11.78. 
(8.5s,+0.8s)	Iteration 10000, current mean episode reward is 11.28. 


In [9]:
# Run this cell without modification

# You should see a pop up window which display the movement of the cart and pole.
print("Average episode reward for your linear agent in CartPole-v0: ",
      linear_trainer.evaluate(1, render=True))

Average episode reward for your linear agent in CartPole-v0:  11.0


In [10]:
# Solve the TODOs and remove `pass`
import math

linear_fc_config = merge_config(dict(
    polynomial_order=1,
), linear_approximator_config)


def polynomial_feature(sequence, order=1):
    """
    Construct the order-n polynomial-basis feature of the state.
    Refer to Chapter 9.5.1 of the textbook. 
    We expect to get a vector of length `(order+1)^k` as the output,
    wherein `k` is the dimensions of the state.

    For example:
    When the state is [2, 3, 4] (so k=3), 
    the first order polynomial feature of the state is 
    [
        1,
        2,
        3,
        4,
        2 * 3 = 6,
        2 * 4 = 8,
        3 * 4 = 12,
        2 * 3 * 4 = 24
    ].
    
    We have `(1+1)^3=8` output dimensions.

    Note: it is not necessary to follow the ascending order.
    """
    # [TODO] finish this function.
    output = []
    if len(sequence)==3:
        for i in range(order+1):
            temp1 = sequence[0]**i
            for j in range(order+1):
                temp2 = temp1*sequence[1]**j
                for k in range(order+1):
                    output.append(temp2*sequence[2]**k)
    if len(sequence)==4:
        for i in range(order+1):
            temp1 = sequence[0]**i
            for j in range(order+1):
                temp2 = temp1*sequence[1]**j
                for k in range(order+1):
                    temp3 = temp2*sequence[2]**k
                    for r in range(order+1):
                        output.append(temp3*sequence[3]**r)      
    return output
  
assert sorted(polynomial_feature([2, 3, 4])) == [1, 2, 3, 4, 6, 8, 12, 24]
assert len(polynomial_feature([2, 3, 4], 2)) == 27
assert len(polynomial_feature([2, 3, 4], 3)) == 64

class LinearTrainerWithFeatureConstruction(LinearTrainer):
    """In this class, we will expand the dimension of the state.
    This procedure is done at self.process_state function.
    The modification of self.obs_dim and the shape of parameters
    is also needed.
    """
    def __init__(self, config):
        config = merge_config(config, linear_fc_config)
        # Initialize the abstract class.
        super().__init__(config)

        self.polynomial_order = self.config["polynomial_order"]

        # Expand the size of observation
        self.obs_dim = (self.polynomial_order + 1) ** self.obs_dim

        # Since we change self.obs_dim, reset the parameters.
        self.initialize_parameters()

    def process_state(self, state):
        """Please finish the polynomial function."""
        processed = polynomial_feature(state, self.polynomial_order)
        processed = np.asarray(processed)
        assert len(processed) == self.obs_dim, processed.shape
        return processed

In [11]:
# Run this cell without modification

linear_fc_trainer, _ = run(LinearTrainerWithFeatureConstruction, dict(
    max_iteration=10000,
    evaluate_interval=1000, 
    parameter_std=0.01,
    learning_rate=0.001,
    polynomial_order=1,
    n=3,
    env_name="CartPole-v0"
), reward_threshold=195.0)

assert linear_fc_trainer.evaluate() > 20.0, "The best episode reward happening " \
    "during training should be greater than the random baseline. That is more than 20+."

# This cell should be finished within 10 minitines.

Initialize parameters with shape: (4, 2).
Initialize parameters with shape: (16, 2).
(0.1s,+0.1s)	Iteration 0, current mean episode reward is 10.94. 
(4.1s,+4.0s)	Iteration 1000, current mean episode reward is 26.74. 
(8.5s,+4.4s)	Iteration 2000, current mean episode reward is 34.38. 
(13.0s,+4.4s)	Iteration 3000, current mean episode reward is 20.1. 
(17.2s,+4.2s)	Iteration 4000, current mean episode reward is 19.34. 
(21.2s,+4.0s)	Iteration 5000, current mean episode reward is 19.8. 
(24.7s,+3.5s)	Iteration 6000, current mean episode reward is 19.98. 
(28.2s,+3.5s)	Iteration 7000, current mean episode reward is 21.54. 
(31.9s,+3.7s)	Iteration 8000, current mean episode reward is 20.0. 
(35.4s,+3.5s)	Iteration 9000, current mean episode reward is 18.0. 
(38.8s,+3.4s)	Iteration 10000, current mean episode reward is 22.8. 


In [12]:
# Run this cell without modification

# You should see a pop up window which display the movement of the cart and pole.
print(
    "In CartPole-v0, the average episode reward for the value estimator "
    "with feature construction is: ",
      linear_fc_trainer.evaluate(1, render=True))

In CartPole-v0, the average episode reward for the value estimator with feature construction is:  11.0


In [13]:
# Solve the TODOs and remove `pass`

# Build the algorithm-specify config.
mlp_trainer_config = merge_config(dict(
    parameter_std=0.01,
    learning_rate=0.01,
    hidden_dim=100,
    n=3,
    clip_norm=1.0,
    clip_gradient=True
), default_config)


class MLPTrainer(LinearTrainer):
    def __init__(self, config):
        config = merge_config(config, mlp_trainer_config)
        self.hidden_dim = config["hidden_dim"]
        super().__init__(config)

    def initialize_parameters(self):
        # [TODO] Initialize self.hidden_parameters and self.output_parameters,
        #  which are two dimensional matrices, and subject to normal
        #  distributions with scale config["parameter_std"]
        std = self.config["parameter_std"]
        self.hidden_parameters = std*np.random.random_sample(size=(self.obs_dim, self.hidden_dim))
        self.output_parameters = std*np.random.random_sample(size=(self.hidden_dim, self.act_dim))
        

    def compute_values(self, processed_state):
        """[TODO] Compute the value for each potential action. Note that you
        should NOT preprocess the state here."""
        assert processed_state.ndim == 1, processed_state.shape
        activation = self.compute_activation(processed_state)
        values = np.dot(self.output_parameters.transpose(),activation)  
        
        return values

    def compute_activation(self, processed_state):
        """[TODO] Compute the action values values.
        Given a processed state, first we need to compute the activtaion
        (the output of hidden layer). Then we compute the values (the output of
        the output layer).
        """
        activation = np.dot(self.hidden_parameters.transpose(),processed_state)

    
        return activation

    def compute_gradient(self, processed_states, actions, rewards, tau, T):
        n = self.n
        
        # [TODO] compute the target value.
        # Hint: copy your codes in LinearTrainer.
        G = 0
        #for t in reversed(range(n)):
            #G = (G**self.gamma)+rewards[t+tau]
        for t in range(tau, tau+n-1):
            G += (self.gamma**(t-tau))*rewards[t]    
            
                  
        if tau + n < T:
            G += (self.gamma**n)*max(self.compute_values(processed_states[tau+n]))

        # Denote the state-action value function Q, then the loss of
        # prediction error w.r.t. the output layer weights can be 
        # separated into two parts (the chain rule):
        #     dError / dweight = (dError / dQ) * (dQ / dweight)
        # We call the first one loss_grad, and the latter one
        # value_grad. We consider the Mean Square Error between the target
        # value (G) and the predict value (Q(s_t, a_t)) to be the loss.
        #cur_state = processed_states[tau]

        loss_grad = np.zeros((self.act_dim, 1))  # [act_dim, 1]
        # [TODO] compute loss_grad
        loss_grad[:,0] = -(G*np.ones(2)-self.compute_values(processed_states[tau]))
        
        # [TODO] compute the gradient of output layer parameters
        value_gradient = np.zeros((self.hidden_dim, 1))
        value_gradient[:,0] = self.compute_activation(processed_states[tau])
        output_gradient = np.dot(value_gradient, loss_grad.transpose())

        
        # [TODO] compute the gradient of hidden layer parameters
        # Hint: using chain rule and derive the formulation
        cur_state = np.zeros((self.obs_dim, 1))
        cur_state[:,0] = processed_states[tau]
        activate_gradient = np.dot(self.output_parameters,loss_grad)      
        hidden_gradient = np.dot(cur_state,activate_gradient.transpose()) 
    
        assert np.all(np.isfinite(output_gradient)), \
            "Invalid value occurs in output_gradient! {}".format(
                output_gradient)
        assert np.all(np.isfinite(hidden_gradient)), \
            "Invalid value occurs in hidden_gradient! {}".format(
                hidden_gradient)
        return [hidden_gradient, output_gradient]

    def apply_gradient(self, gradients):
        """Apply the gradientss to the two layers' parameters."""
        assert len(gradients) == 2
        hidden_gradient, output_gradient = gradients

        assert output_gradient.shape == (self.hidden_dim, self.act_dim)
        assert hidden_gradient.shape == (self.obs_dim, self.hidden_dim)
        
        # [TODO] Implement the clip gradient mechansim
        # Hint: when the old gradient has norm less that clip_norm,
        #  then nothing happens. Otherwise shrink the gradient to
        #  make its norm equal to clip_norm.
        if self.config["clip_gradient"]:
            clip_norm = self.config["clip_norm"]
            #import pdb; pdb.set_trace()
            if np.linalg.norm(output_gradient,2)>clip_norm:
                output_gradient = output_gradient*clip_norm/np.linalg.norm(output_gradient,2)
            if np.linalg.norm(hidden_gradient,2)>clip_norm:
                hidden_gradient = hidden_gradient*clip_norm/np.linalg.norm(hidden_gradient,2)

        # [TODO] update the parameters
        # Hint: Remember to check the sign when applying the gradient
        #  into the parameters. Should you add or minus the gradients?
        self.learning_rate = self.config["learning_rate"]
        self.hidden_parameters -= self.learning_rate*hidden_gradient
        self.output_parameters -= self.learning_rate*output_gradient  

In [14]:
# Run this cell without modification

print("Now let's see what happen if gradient clipping is not enable!\n")
try:
    failed_mlp_trainer, _ = run(MLPTrainer, dict(
        max_iteration=3000,
        evaluate_interval=100, 
        parameter_std=0.01,
        learning_rate=0.001,
        hidden_dim=100,
        clip_gradient=False,  # <<< Gradient clipping is OFF!
        env_name="CartPole-v0"
    ), reward_threshold=195.0)
    print("\nWe expect to see bad performance (<195). "
          "The performance without gradient clipping: {}."
          "".format(failed_mlp_trainer.evaluate()))
except AssertionError as e:
    print(traceback.format_exc())
    print("Infinity happen during training. It's OK since the gradient is not bounded.")
finally:
    print("Try next cell to see the impact of gradient clipping.")

Now let's see what happen if gradient clipping is not enable!

(0.0s,+0.0s)	Iteration 0, current mean episode reward is 11.34. 
(0.4s,+0.3s)	Iteration 100, current mean episode reward is 11.74. 
(0.5s,+0.1s)	Iteration 200, current mean episode reward is 11.2. 
(0.6s,+0.1s)	Iteration 300, current mean episode reward is 11.78. 
(0.8s,+0.1s)	Iteration 400, current mean episode reward is 11.12. 
(0.9s,+0.1s)	Iteration 500, current mean episode reward is 10.6. 
(1.0s,+0.1s)	Iteration 600, current mean episode reward is 13.34. 
(1.2s,+0.1s)	Iteration 700, current mean episode reward is 11.92. 
(1.3s,+0.1s)	Iteration 800, current mean episode reward is 10.94. 
(1.6s,+0.3s)	Iteration 900, current mean episode reward is 50.04. 
(1.8s,+0.2s)	Iteration 1000, current mean episode reward is 11.02. 
(1.9s,+0.1s)	Iteration 1100, current mean episode reward is 11.32. 
(2.0s,+0.1s)	Iteration 1200, current mean episode reward is 11.04. 
(2.2s,+0.2s)	Iteration 1300, current mean episode reward is 11.38. 

In [15]:
# Run this cell without modification

print("Now let's see what happen if gradient clipping is enable!\n")
mlp_trainer, _ = run(MLPTrainer, dict(
    max_iteration=3000,
    evaluate_interval=100, 
    parameter_std=0.01,
    learning_rate=0.001,
    hidden_dim=100,
    clip_gradient=True,  # <<< Gradient clipping is ON!
    env_name="CartPole-v0"
), reward_threshold=195.0)

assert mlp_trainer.evaluate() > 195.0, "Check your codes. " \
    "Your agent should achieve {} reward in 200 iterations." \
    "But it achieve {} reward in evaluation."

# In our implementation, the task is solved in 200 iterations.

Now let's see what happen if gradient clipping is enable!

(0.1s,+0.1s)	Iteration 0, current mean episode reward is 11.34. 
(0.6s,+0.5s)	Iteration 100, current mean episode reward is 11.74. 
(0.8s,+0.2s)	Iteration 200, current mean episode reward is 11.2. 
(1.0s,+0.2s)	Iteration 300, current mean episode reward is 11.78. 
(1.2s,+0.2s)	Iteration 400, current mean episode reward is 11.12. 
(1.4s,+0.2s)	Iteration 500, current mean episode reward is 10.6. 
(1.6s,+0.2s)	Iteration 600, current mean episode reward is 13.34. 
(1.8s,+0.2s)	Iteration 700, current mean episode reward is 11.92. 
(2.0s,+0.2s)	Iteration 800, current mean episode reward is 10.94. 
(2.5s,+0.4s)	Iteration 900, current mean episode reward is 50.04. 
(2.7s,+0.3s)	Iteration 1000, current mean episode reward is 11.02. 
(2.9s,+0.2s)	Iteration 1100, current mean episode reward is 11.32. 
(3.1s,+0.2s)	Iteration 1200, current mean episode reward is 11.04. 
(3.3s,+0.2s)	Iteration 1300, current mean episode reward is 11.38. 
(3.

AssertionError: Check your codes. Your agent should achieve {} reward in 200 iterations.But it achieve {} reward in evaluation.

In [16]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [17]:
# Solve the TODOs and remove `pass`

from collections import deque
import random

class ExperienceReplayMemory:
    """Store and sample the transitions"""
    def __init__(self, capacity):
        # deque is a useful class which acts like a list but only contain
        # finite elements.When appending new element make deque exceeds the 
        # `maxlen`, the oldest element (the index 0 element) will be removed.
        
        # [TODO] uncomment next line. 
        self.memory = deque(maxlen=capacity)
        pass

    def push(self, transition):
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [18]:
# Solve the TODOs and remove `pass`

class PytorchModel(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(PytorchModel, self).__init__()
        
        # [TODO] Build a sequential model with two layers.
        # The first hidden layer has 100 hidden nodes, followed by
        # a ReLU activation function.
        # The second output layer take the activation vector, who has
        # 100 elements, as input and return the action values.
        # So the return values is a vector with num_actions elements.
        self.input_shape = input_shape
        self.num_actions = num_actions
        self.num_hidden = 100
        
        self.hidden_value = nn.Linear(self.input_shape[0], self.num_hidden)
        self.action_value = nn.Linear(self.num_hidden, self.num_actions)
        

    def forward(self, obs):
        obs = F.relu(self.hidden_value(obs))
        return self.action_value(obs)
    
# Test
assert isinstance(PytorchModel((3,), 7).action_value, nn.Module)

In [19]:
# Solve the TODOs and remove `pass`

pytorch_config = merge_config(dict(
    memory_size=50000,
    learn_start=5000,
    batch_size=32,
    target_update_freq=500,  # in steps
    learn_freq=1,  # in steps
    n=1
), mlp_trainer_config)


def to_tensor(x):
    """A helper function to transform a numpy array to a Pytorch Tensor"""
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x).type(torch.float32)
    assert isinstance(x, torch.Tensor)
    if x.dim() == 3 or x.dim() == 1:
        x = x.unsqueeze(0)
    assert x.dim() == 2 or x.dim() == 4, x.shape
    return x


class DQNTrainer(MLPTrainer):
    def __init__(self, config):
        config = merge_config(config, pytorch_config)
        self.learning_rate = config["learning_rate"]
        super().__init__(config)

        self.memory = ExperienceReplayMemory(config["memory_size"])
        self.learn_start = config["learn_start"]
        self.batch_size = config["batch_size"]
        self.target_update_freq = config["target_update_freq"]
        self.clip_norm = config["clip_norm"]
        self.step_since_update = 0
        self.total_step = 0

    def initialize_parameters(self):
        input_shape = self.env.observation_space.shape
        self.num_actions = self.env.action_space.n

        # [TODO] Initialize two network using PytorchModel class
        self.network = PytorchModel(input_shape, self.num_actions)

        self.network.eval()
        self.network.share_memory()

        # [TODO] Initialize target network then copy the weight
        # of original network to it. So you should
        # put the weights of self.network into self.target_network.
        self.target_network = PytorchModel(input_shape, self.num_actions)
        
        self.target_network.eval()

        # Build Adam optimizer and MSE Loss.
        # [TODO] Uncomment next few lines
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=self.learning_rate)
        self.loss = nn.MSELoss()
        
    def compute_values(self, processed_state):
        """Compute the value for each potential action. Note that you
        should NOT preprocess the state here."""
        # [TODO] Convert the output of neural network to numpy array
        params = list(self.network.parameters())
        self.hidden_parameters = params[0].detach().numpy()
        self.hidden_parameters_b = params[1].detach().numpy()
        self.output_parameters = params[2].detach().numpy()
        self.output_parameters_b = params[3].detach().numpy()
        #import pdb; pdb.set_trace()                
        activation = np.dot(self.hidden_parameters,processed_state.numpy())+self.hidden_parameters_b
        values = np.dot(self.output_parameters,activation)+self.output_parameters_b
    
        return values

    def train(self):
        s = self.env.reset()
        processed_s = self.process_state(s)
        act = self.compute_action(processed_s)
        stat = {"loss": []}

        for t in range(self.max_episode_length):
            next_state, reward, done, _ = self.env.step(act)
            next_processed_s = self.process_state(next_state)

            # Push the transition into memory.
            self.memory.push(
                (processed_s, act, reward, next_processed_s, done)
            )

            processed_s = next_processed_s
            act = self.compute_action(next_processed_s)
            self.step_since_update += 1
            self.total_step += 1

            if done:
                break
                
            if t % self.config["learn_freq"] != 0:
                # It's not necessary to update in each step.
                continue

            if len(self.memory) < self.learn_start:
                continue
            elif len(self.memory) == self.learn_start:
                print("Current memory contains {} transitions, "
                      "start learning!".format(self.learn_start))

            batch = self.memory.sample(self.batch_size)

            # Transform a batch of state / action / .. into a tensor.
            state_batch = to_tensor(
                np.stack([transition[0] for transition in batch])
            )
            action_batch = to_tensor(
                np.stack([transition[1] for transition in batch])
            )
            reward_batch = to_tensor(
                np.stack([transition[2] for transition in batch])
            )
            next_state_batch = torch.stack(
                [transition[3] for transition in batch]
            )
            done_batch = to_tensor(
                np.stack([transition[4] for transition in batch])
            )

            with torch.no_grad():
                # [TODO] Compute the values of Q in next state in batch.
                Q_t_plus_one = self.target_network(next_state_batch).max(1)[0].detach()
               
                assert isinstance(Q_t_plus_one, torch.Tensor)
                assert Q_t_plus_one.dim() == 1
                
                # [TODO] Compute the target value of Q in batch.
                Q_target = reward_batch+0.99*Q_t_plus_one
                #import pdb; pdb.set_trace() 
                #assert Q_target.shape == (self.batch_size,)
            
            # [TODO] Collect the Q values in batch.
            # Hint: Remember to call self.network.train()
            #  before you get the Q value from self.network(state_batch),
            #  otherwise the graident will not be recorded by pytorch.
            self.network.train()
            #import pdb; pdb.set_trace() 
            Q_t = self.network(state_batch).gather(1, action_batch.long())
    
            assert Q_t.shape == Q_target.shape

            # Update the network
            self.optimizer.zero_grad()
            loss = self.loss(input=Q_t, target=Q_target)
            loss_value = loss.item()
            stat['loss'].append(loss_value)
            loss.backward()
            
            # [TODO] Gradient clipping. Uncomment next line
            nn.utils.clip_grad_norm_(self.network.parameters(), self.clip_norm)
            
            self.optimizer.step()
            self.network.eval()

        if len(self.memory) >= self.learn_start and \
                self.step_since_update > self.target_update_freq:
            print("{} steps has passed since last update. Now update the"
                  " parameter of the behavior policy. Current step: {}".format(
                self.step_since_update, self.total_step
            ))
            self.step_since_update = 0
            # [TODO] Copy the weights of self.network to self.target_network.
            self.target_network.load_state_dict(self.network.state_dict())
            
            self.target_network.eval()
            
        return {"loss": np.mean(stat["loss"]), "episode_len": t}

    def process_state(self, state):
        return torch.from_numpy(state).type(torch.float32)

In [20]:
# Run this cell without modification

# Build the test trainer.
test_trainer = DQNTrainer({})

# Test compute_values
fake_state = test_trainer.env.observation_space.sample()
processed_state = test_trainer.process_state(fake_state)
assert processed_state.shape == (test_trainer.obs_dim, ), processed_state.shape
values = test_trainer.compute_values(processed_state)
assert values.shape == (test_trainer.act_dim, ), values.shape

test_trainer.train()
print("Now your codes should be bug-free.")

_ = run(DQNTrainer, dict(
    max_iteration=20,
    evaluate_interval=10, 
    learn_start=100,
    env_name="CartPole-v0",
))

print("Test passed!")

D:\Softwares\Anaconda\envs\ierg5350\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Softwares\Anaconda\envs\ierg5350\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Now your codes should be bug-free.
(0.1s,+0.1s)	Iteration 0, current mean episode reward is 10.56. {'loss': nan, 'episode_len': 9.0}
Current memory contains 100 transitions, start learning!


D:\Softwares\Anaconda\envs\ierg5350\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


(1.2s,+1.1s)	Iteration 10, current mean episode reward is 11.38. {'loss': 0.1068, 'episode_len': 16.0}
(1.5s,+0.3s)	Iteration 20, current mean episode reward is 11.5. {'loss': 0.0518, 'episode_len': 8.0}
Test passed!


In [21]:
# Run this cell without modification

pytorch_trainer, pytorch_stat = run(DQNTrainer, dict(
    max_iteration=2000,
    evaluate_interval=10, 
    learning_rate=0.01,
    clip_norm=10.0,
    memory_size=50000,
    learn_start=1000,
    eps=0.1,
    target_update_freq=1000,
    batch_size=32,
    env_name="CartPole-v0",
), reward_threshold=195.0)

reward = pytorch_trainer.evaluate()
assert reward > 195.0, "Check your codes. " \
    "Your agent should achieve {} reward within 1000 iterations." \
    "But it achieve {} reward in evaluation.".format(195.0, reward)

# Should solve the task in 10 minutes

(0.0s,+0.0s)	Iteration 0, current mean episode reward is 9.78. {'loss': nan, 'episode_len': 9.0}
(0.1s,+0.0s)	Iteration 10, current mean episode reward is 9.76. {'loss': nan, 'episode_len': 7.0}
(0.1s,+0.0s)	Iteration 20, current mean episode reward is 9.88. {'loss': nan, 'episode_len': 8.0}
(0.2s,+0.0s)	Iteration 30, current mean episode reward is 9.6. {'loss': nan, 'episode_len': 10.0}
(0.2s,+0.0s)	Iteration 40, current mean episode reward is 9.74. {'loss': nan, 'episode_len': 10.0}
(0.3s,+0.1s)	Iteration 50, current mean episode reward is 9.62. {'loss': nan, 'episode_len': 9.0}
(0.3s,+0.0s)	Iteration 60, current mean episode reward is 9.64. {'loss': nan, 'episode_len': 8.0}
(0.4s,+0.0s)	Iteration 70, current mean episode reward is 9.6. {'loss': nan, 'episode_len': 9.0}
(0.4s,+0.0s)	Iteration 80, current mean episode reward is 9.98. {'loss': nan, 'episode_len': 8.0}
(0.5s,+0.1s)	Iteration 90, current mean episode reward is 9.9. {'loss': nan, 'episode_len': 8.0}
(0.5s,+0.1s)	Iteration

(15.8s,+0.3s)	Iteration 730, current mean episode reward is 9.84. {'loss': 0.0081, 'episode_len': 9.0}
(16.1s,+0.3s)	Iteration 740, current mean episode reward is 9.9. {'loss': 0.001, 'episode_len': 9.0}
(16.4s,+0.3s)	Iteration 750, current mean episode reward is 9.9. {'loss': 0.0014, 'episode_len': 10.0}
1013 steps has passed since last update. Now update the parameter of the behavior policy. Current step: 8049
(16.7s,+0.3s)	Iteration 760, current mean episode reward is 9.86. {'loss': 0.5402, 'episode_len': 24.0}
(16.9s,+0.3s)	Iteration 770, current mean episode reward is 9.88. {'loss': 0.0172, 'episode_len': 8.0}
(17.1s,+0.2s)	Iteration 780, current mean episode reward is 10.1. {'loss': 0.0188, 'episode_len': 10.0}
(17.4s,+0.3s)	Iteration 790, current mean episode reward is 9.9. {'loss': 0.0182, 'episode_len': 12.0}
(17.7s,+0.3s)	Iteration 800, current mean episode reward is 10.12. {'loss': 0.017, 'episode_len': 8.0}
(17.9s,+0.3s)	Iteration 810, current mean episode reward is 10.14. 

(38.2s,+0.2s)	Iteration 1410, current mean episode reward is 10.06. {'loss': 0.0293, 'episode_len': 9.0}
(38.4s,+0.2s)	Iteration 1420, current mean episode reward is 10.06. {'loss': 0.0027, 'episode_len': 9.0}
(38.6s,+0.2s)	Iteration 1430, current mean episode reward is 10.02. {'loss': 0.0018, 'episode_len': 8.0}
(38.8s,+0.2s)	Iteration 1440, current mean episode reward is 10.14. {'loss': 0.002, 'episode_len': 8.0}
(39.1s,+0.2s)	Iteration 1450, current mean episode reward is 9.82. {'loss': 0.0053, 'episode_len': 9.0}
(39.3s,+0.3s)	Iteration 1460, current mean episode reward is 9.86. {'loss': 0.0025, 'episode_len': 11.0}
(39.6s,+0.2s)	Iteration 1470, current mean episode reward is 10.12. {'loss': 0.0025, 'episode_len': 11.0}
(39.8s,+0.2s)	Iteration 1480, current mean episode reward is 10.06. {'loss': 0.0019, 'episode_len': 10.0}
(40.0s,+0.2s)	Iteration 1490, current mean episode reward is 9.86. {'loss': 0.0037, 'episode_len': 9.0}
1002 steps has passed since last update. Now update the 

AssertionError: Check your codes. Your agent should achieve 195.0 reward within 1000 iterations.But it achieve 10.04 reward in evaluation.

In [22]:
# Run this cell without modification

# You should see a pop up window which display the movement of the cart and pole.
print("Average episode reward for your Pytorch agent in CartPole-v0: ",
      pytorch_trainer.evaluate(1, render=True))

Average episode reward for your Pytorch agent in CartPole-v0:  11.0
